In [17]:
import pandas as pd

In [18]:
# from google.colab import drive
# drive.mount('/content/drive')

In [68]:
from konlpy.tag import Komoran
import pickle

class Preprocess:

  def __init__(self,userdic=None):

    self.komoran = Komoran(userdic=userdic)

    self.exclusion_tags = [
    'JKS', 'JKC', 'JKG', 'JKO', 'JKB', 'JKV', 'JKQ',
    # 주격조사, 보격조사, 관형격조사, 목적격조사, 부사격조사, 호격조사, 인용격조사
    'JX', 'JC',
    # 보조사, 접속조사
    'SF', 'SP', 'SS', 'SE', 'SO',
    # 마침표,물음표,느낌표(SF), 쉼표,가운뎃점,콜론,빗금(SP), 따옴표,괄호표,줄표(SS), 줄임표(SE), 붙임표(물결,숨김,빠짐)(SO)
    'EP', 'EF', 'EC', 'ETN', 'ETM',
    # 선어말어미, 종결어미, 연결어미, 명사형전성어미, 관형형전성어미
    'XSN', 'XSV', 'XSA'
    # 명사파생접미사, 동사파생접미사, 형용사파생접미사
    ]
        
    # 형태소 분석기 POS 태거
  def pos(self, sentence):
    return self.komoran.pos(sentence)

    # 불용어 제거 후 필요한 품사 정보만 가져오기
  def get_keywords(self, pos, without_tag=False):
    f = lambda x: x in self.exclusion_tags
    print(pos)
    word_list = []
    for p in pos:
        if f(p[1]) is False:
            word_list.append(p if without_tag is False else p[0])
    return word_list



In [69]:
sent = "컴공 과사 번호를 알려줘 병신아!"

# userdic 
# https://needjarvis.tistory.com/741
# 직접 우리 프로젝트 테마에 맞게 등록하면됌
# ex)
# 짜장면 NNG

# 전처리 객체
p = Preprocess(userdic='/content/drive/MyDrive/새싹_인공지능SW교육/프로젝트/새싹_최종프로젝트/chatbot_try/useridc/user_dic.tsv')

# 형태소 분석기
pos = p.pos(sent)

# 키워드,품사 출력
ret = p.get_keywords(pos,without_tag=False)
ret

[('컴공', 'NNP'), ('과사', 'NNP'), ('번호', 'NNG'), ('를', 'JKO'), ('알리', 'VV'), ('어', 'EC'), ('주', 'VX'), ('어', 'EC'), ('병신', 'NNP'), ('아', 'JKV'), ('!', 'SF')]


[('컴공', 'NNP'),
 ('과사', 'NNP'),
 ('번호', 'NNG'),
 ('알리', 'VV'),
 ('주', 'VX'),
 ('병신', 'NNP')]

In [78]:
import pandas as pd
import pickle

# 말뭉치 데이터 읽어오기
movie_review = pd.read_csv('/content/drive/MyDrive/새싹_인공지능SW교육/프로젝트/새싹_최종프로젝트/chatbot_try/csv_data/영화리뷰.csv',encoding='CP949')
purpose = pd.read_csv('/content/drive/MyDrive/새싹_인공지능SW교육/프로젝트/새싹_최종프로젝트/chatbot_try/csv_data/용도별목적대화데이터.csv',encoding='CP949')
common_sense = pd.read_csv('/content/drive/MyDrive/새싹_인공지능SW교육/프로젝트/새싹_최종프로젝트/chatbot_try/csv_data/일반상식.csv',encoding='CP949')
topic = pd.read_csv('/content/drive/MyDrive/새싹_인공지능SW교육/프로젝트/새싹_최종프로젝트/chatbot_try/csv_data/주제별일상대화데이터.csv',encoding='CP949')

In [79]:
topic.head()

,text
0,애덜앙 나 너무 배불러서 배 아파
1,배 아프면 약국 가서 소화제라도 마시는 거 어때
2,얼마나 먹었다고 배가 아프데
3,떡볶이랑 튀김이랑 닭껍질튀김이랑 마카롱 먹었어
4,하긴 *이 배는 파리보다 작으니까 당연히 그만큼 먹어도 배부르지


In [80]:
common_sense.head()

,intent,query,answer
0,일반상식,다테 기미코가 최초로 은퇴 선언을 한게 언제지,1996년 9월 24일
1,일반상식,ave;new 본거지 어디야,도쿄 치요다구
2,일반상식,사카이 다다요시의 아버지가 누구지,사카이 나오타카(酒井直隆)
3,일반상식,일반성면의 면적이 얼마야,19.41 km²
4,일반상식,금나라를 세운 사람이 누구야,태조 아구다


In [81]:
# 결측값 제거
movie_review.dropna(inplace=True)
purpose.dropna(inplace=True)
topic.dropna(inplace=True)
common_sense.dropna(inplace=True)

In [82]:
text1 = list(movie_review['document'])
text2 = list(purpose['text'])
text3 = list(topic['text'])
text4 = list(common_sense['query'])+list(common_sense['answer'])

In [84]:
corpus_data = text1 + text2 + text3 + text4

In [87]:
from tqdm import tqdm

# 말뭉치 데이터에서 키워드만 추출하여 사전 리스트 생성
# 약 20분 소요
p = Preprocess()

dict1 = []
for c in tqdm(corpus_data):
  pos = p.pos(c)
  for k in pos:
    dict1.append(k[0])

dict1[:50]

100%|██████████| 2263322/2263322 [17:59<00:00, 2096.20it/s]


['어리',
 'ㄹ',
 '때',
 '보고',
 '지금',
 '다시',
 '보',
 '아도',
 '재밌어요ㅋㅋ',
 '디자인',
 '을',
 '배우',
 '는',
 '학생',
 '으로',
 ',',
 '외국',
 '디자이너',
 '와',
 '그',
 '들',
 '이',
 '일구',
 'ㄴ',
 '전통',
 '을',
 '통하',
 '아',
 '발전',
 '하',
 '아',
 '가',
 '는',
 '문화',
 '산업',
 '이',
 '부럽',
 '었',
 '는데',
 '.',
 '사실',
 '우리나라',
 '에서',
 '도',
 '그',
 '어렵',
 'ㄴ',
 '시절',
 '에',
 '끝']

In [88]:
# 사전에 사용될 word2index 생성
# 사전의 첫번쨰 인덱스에는 OOV 사용
# 사전에 없는 단어는 word2index = 1로 바꿔줌

# OOV token 이란? https://hyoeun-log.tistory.com/entry/WEEK1-NLP-in-Tensorflow-Tokenizer-OOV-token-padsequences

tokenizer = preprocessing.text.Tokenizer(oov_token='OOV', num_words=100000)
tokenizer.fit_on_texts(dict1)
word_index = tokenizer.word_index
print(len(word_index))

157410


In [89]:
# bin 파일로 만들면 데이터 전송 시 빠르다(?)
f = open("chatbot_dict.bin","wb")
try:
  pickle.dump(word_index,f)
except Exception as e:
  print(e)
finally:
  f.close()

In [92]:
f = open("/content/drive/MyDrive/새싹_인공지능SW교육/프로젝트/새싹_최종프로젝트/chatbot_try/chatbot_dict.bin","rb")
word_index = pickle.load(f)
f.close()

# 확인1

# 전처리 객체 생성
p = Preprocess(userdic='/content/drive/MyDrive/새싹_인공지능SW교육/프로젝트/새싹_최종프로젝트/chatbot_try/useridc/user_dic.tsv')

# 예문
sent1 = "안녕하세요.테스트 코드 입니다. 단어사전 만들기 성공?"

# 형태소 분석
pos = p.pos(sent1)

# 품사 태그없이 키워드 출력

keywords = p.get_keywords(pos,without_tag=True)
for word in keywords:
  try:
    print(word,word_index[word])
  except KeyError:
    # key없을 때 key 에러(단어가 사전에 없을때)
    # OOV 처리됌
    print(word,wird_index['OOV'])

[('안녕하세요', 'NNP'), ('.', 'SF'), ('테스트', 'NNP'), ('코드', 'NNP'), ('이', 'VCP'), ('ㅂ니다', 'EF'), ('.', 'SF'), ('단어', 'NNG'), ('사전', 'NNP'), ('만들', 'VV'), ('기', 'ETN'), ('성공', 'NNG'), ('?', 'SF')]
안녕하세요 137
테스트 2438
코드 1907
이 3
단어 1941
사전 2030
만들 164
성공 1275


In [96]:
# 확인2

# 전처리 객체 생성
p = Preprocess(userdic='/content/drive/MyDrive/새싹_인공지능SW교육/프로젝트/새싹_최종프로젝트/chatbot_try/useridc/user_dic.tsv')

# 예문
sent2 = "최종 프로젝트 잘 마치고 취업합시다! 화이팅! 취뽀 가자! 오늘 점심 호호반점 갈까?"

# 형태소 분석
pos = p.pos(sent2)

# 품사 태그없이 키워드 출력

keywords = p.get_keywords(pos,without_tag=True)
for word in keywords:
  try:
    print(word,word_index[word])
  except KeyError:
    # key없을 때 key 에러(단어가 사전에 없을때)
    # OOV 처리됌
    print(word,word_index['OOV'])

[('최종', 'NNG'), ('프로젝트', 'NNP'), ('잘', 'MAG'), ('마치', 'VV'), ('고', 'EC'), ('취업', 'NNG'), ('하', 'XSV'), ('ㅂ시다', 'EF'), ('!', 'SF'), ('화이팅', 'NNP'), ('!', 'SF'), ('취뽀', 'NA'), ('가', 'VV'), ('자', 'EF'), ('!', 'SF'), ('오늘', 'NNG'), ('점심', 'NNP'), ('호', 'NNB'), ('호반', 'NNP'), ('점', 'NNB'), ('가', 'VV'), ('ㄹ까', 'EF'), ('?', 'SF')]
최종 3560
프로젝트 3559
잘 61
마치 1864
취업 959
화이팅 2058
취뽀 40470
가 8
오늘 165
점심 1031
호 1188
호반 18172
점 176
가 8


In [ ]:
# 전처리 클래스에
#  단어 인덱스 사전 추가함

class Preprocess:
    def __init__(self, word2index_dic='../train_tools/dict/chatbot_dict.bin' ,userdic='user_dic.tsv'): # userdic 인자에는 사용자 정의 사전 파일 경로 입력가능
        # 단어 인덱스 사전 불러오기 추가
        if(word2index_dic != ''):
            f = open(word2index_dic, "rb")
            self.word_index = pickle.load(f)
            f.close()
            print("단어 사전 로드 완료..")
        else:
            self.word_index = None
            print("단어 사전 로드 실패..")

        # 형태소 분석기 초기화
        self.komoran = Komoran(userdic=userdic)

        # 제외할 품사
        # 참조 : https://docs.komoran.kr/firststep/postypes.html
        # 관계언 제거, 기호 제거
        # 어미 제거
        # 접미사 제거
        self.exclusion_tags = [
            'JKS', 'JKC', 'JKG', 'JKO', 'JKB', 'JKV', 'JKQ',
            # 주격조사, 보격조사, 관형격조사, 목적격조사, 부사격조사, 호격조사, 인용격조사
            'JX', 'JC',
            # 보조사, 접속조사
            'SF', 'SP', 'SS', 'SE', 'SO',
            # 마침표,물음표,느낌표(SF), 쉼표,가운뎃점,콜론,빗금(SP), 따옴표,괄호표,줄표(SS), 줄임표(SE), 붙임표(물결,숨김,빠짐)(SO)
            'EP', 'EF', 'EC', 'ETN', 'ETM',
            # 선어말어미, 종결어미, 연결어미, 명사형전성어미, 관형형전성어미
            'XSN', 'XSV', 'XSA'
            # 명사파생접미사, 동사파생접미사, 형용사파생접미사
        ]


    # 형태소 분석기 POS 태거
    def pos(self, sentence):
        return self.komoran.pos(sentence)

    # 불용어 제거 후 필요한 품사 정보만 가져오기
    def get_keywords(self, pos, without_tag=False):
        f = lambda x: x in self.exclusion_tags
        word_list = []
        for p in pos:
            if f(p[1]) is False:
                word_list.append(p if without_tag is False else p[0])
        return word_list

    # 키워드를 단어 인덱스 시퀀스로 변환
    def get_wordidx_sequence(self, keywords):
        if self.word_index is None:
            return []
        w2i = []
        for word in keywords:
            try:
                w2i.append(self.word_index[word])
            except KeyError:
                # 해당 단어가 사전에 없는 경우 OOV 처리
                w2i.append(self.word_index['OOV'])
        return w2i